# Imports

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# config

In [ ]:
months_data_needed = ['08', '09']

# Data

In [ ]:
df = pd.read_csv('../tweets.csv', header=0)

# Functions

In [ ]:
def tf_idf(data, stopwords=None, max_features=1000, ngram=(2, 3)):
    tf_Idf = TfidfVectorizer(stop_words=stopwords, max_features=max_features, ngram_range=ngram)
    tf_Idf_fit = tf_Idf.fit_transform(data)
    feature_names = tf_Idf.get_feature_names()
    
    dense = tf_Idf_fit.todense()
    denselist = dense.tolist()

    tf_idf_table = pd.DataFrame(denselist, columns=feature_names)

    tf_idf_result = pd.DataFrame()
    tf_idf_result[['topic', 'score']] = tf_idf_table.mean().reset_index()
    tf_idf_result = tf_idf_result.sort_values('score', ascending=False).reset_index(drop=True)
    return tf_idf_result
        
numeric = re.compile(r'\w*\d+\w*')
def accepted_word(w):
    return len(w) > 2 and re.search(numeric, w) is None

# Filter Data

In [ ]:
data = df.text

In [ ]:
data_removed = data.apply(lambda s: " ".join([w for w in s.split() if not accepted_word(w)]))
stopwords = " ".join(data_removed).split()

In [ ]:
len(stopwords)

# get topics

In [ ]:
mo7afazat = df.groupby(['MOHAFAZA_AR', 'month'])
for m_key, m_data in mo7afazat:
    if m_key[0] in months_data_needed:
        topics = tf_idf(m_data.text, stopwords=stopwords, ngram=(2, 3))
        file = m_key[0] + '_' + m_key[1] + '.csv'
        topics.to_csv(file, index=False)
        print('[Saved Data]:', file)

In [ ]:
kadaas = df.groupby(['KADAA_AR', 'month'])
for k_key, k_data in kadaas:
    if k_key[0] in months_data_needed:
        topics = tf_idf(k_data.text, stopwords=stopwords, ngram=(2, 3))
        file = k_key[0] + '_' + k_key[1] + '.csv'
        topics.to_csv(file, index=False)
        print('[Saved Data]:', file)